<a href="https://colab.research.google.com/github/RosendOcana/AlgoritmosDeML/blob/main/Permeabilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hay que subir la carpeta completa de archivos de este [link](https://drive.google.com/drive/folders/1AHL0hQwldsxGREi-jUYnJA-kMkkjtZga?usp=sharing) para que las rutas coincidan, la línea de código:

```
drive.mount('/content/drive')
```
Le pedirá permisos para acceder a su drive y llamar los archivos perme.csv y registros_prueba.csv


In [ ]:
from csv import DictReader
import numpy as np
import numpy.lib.recfunctions as rfn
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
reader = DictReader(open('/content/drive/MyDrive/Permeabilidad/perme.csv'))
dtype = np.dtype([('Prof','f4'),('Poro','f4'),('Perme','f4')])
datos_duros = np.array([tuple([val for key,val in dic.items()]) for dic in reader],dtype=dtype)

In [ ]:
reader_2 = DictReader(open('/content/drive/MyDrive/Permeabilidad/registros_prueba.csv'))
dtype_re = np.dtype([(name,'f4') for name in reader_2.fieldnames])
registros = np.array([tuple([val for key,val in dic.items()]) for dic in reader_2],dtype=dtype_re)

In [ ]:
#Escalamiento del GR
def Rescalar(Curva:np.ndarray,rangos:dict) -> np.ndarray:
    """
    Rescala de manera lineal, utilizando los rangos, cualquier curva de entrada.\n
    Primero convertirá los rangos p1 y p2 a logaritmo para calcular la pendiente.
    :param Curva: Numpy Array Curva a rescalar (GR,DT,RHOB,NPHI...)
    :param rangos: dict rangos para rescalar
    >>> rangos = {'g1':50,'g2':100,'p1':0.01,'p2':1000}
    """
    pendiente = (np.log10(rangos['p2'])-np.log10(rangos['p1'])) / (rangos['g1']- rangos['g2'])
    intercepto = np.log10(rangos['p2']) - rangos['g1']*pendiente
    print(f"Pendiente: {pendiente} | Intercepto: {intercepto}")
    return 10**(intercepto + Curva*pendiente)

GR_escalado = Rescalar(registros['GR'],dict(p1=0.01,p2=1000,g1=50,g2=100))


Pendiente: -0.1 | Intercepto: 8.0


In [ ]:
def Añadir(datos_duros:np.ndarray,registros:np.ndarray,curva:str,duro:str) -> np.ndarray:
    """
    Creará una curva con los datos de Nucleo utilizando las profundidades de registro
    """
    registros = rfn.append_fields(registros,curva,np.full_like(registros['DEPTH'],np.nan),dtypes='f4')
    Profundides = np.unique(np.round(datos_duros['Prof']))
    for idx,prof in enumerate(Profundides):
        registros[curva][registros['DEPTH']==prof] = datos_duros[duro][idx]
    return np.ma.getdata(registros)
registros = Añadir(datos_duros,registros,'PermNuc','Perme')
registros = Añadir(datos_duros,registros,'PoroNuc','Poro')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=2,cols=2,shared_xaxes=True,specs=[[{"secondary_y": True},{"secondary_y": True}],
[{"colspan": 2},None]],subplot_titles=('Gamma no invertido','Gamma invertido'))
fig.add_trace(go.Scatter(x=registros['DEPTH'],y=registros['GR'],name='Gamma',legendgroup='group1',line_color='green'),row=1,col=1)
fig.add_trace(go.Scatter(x=registros['DEPTH'],y=registros['PermNuc'],name='Nucleo',legendgroup='group2',mode='markers',marker={'color':'red'},yaxis='y2'),row=1,col=1,secondary_y=True)
#
fig.add_trace(go.Scatter(x=registros['DEPTH'],y=GR_escalado,legendgroup='group1',showlegend=False,line_color='green'),row=1,col=2)
fig.add_trace(go.Scatter(x=registros['DEPTH'],y=registros['PermNuc'],legendgroup='group2',showlegend=False,mode='markers',marker={'color':'red'},yaxis='y3'),row=1,col=2,secondary_y=True)
#
fig.add_trace(go.Scatter(x=registros['PoroNuc'],y=registros['PermNuc'],mode='markers',showlegend=False,legendgroup='group2',marker={'color':'red'}),row=2,col=1)
fig.add_trace(go.Scatter(x=registros['PoroNuc'],y=GR_escalado,mode='markers',legendgroup='group2',marker={'color':'yellow'},name='Propagados'),row=2,col=1)
#
fig.update_layout(yaxis2={'type':'log','range':[-3,3]},yaxis3={'type':'log','range':[-3,3]},yaxis4={'type':'log','range':[-3,3]},yaxis5={'type':'log','range':[-3,3]})
fig.show()

In [ ]:
from abc import abstractmethod,ABC
class Regresion(ABC):
    def __init__(self,X:np.ndarray,y:np.ndarray) -> None:
        if X.ndim < 2:
            print(f'El array tiene forma {X.shape}, haciendo reshape...')
            self.X = X.reshape(-1,1)
        else:
            self.X = X
            
        if y.ndim < 2:
            self.y = y.reshape(-1,1)
        else:
            self.y = y

    def descenso(self,iteraciones:int=500,lr:int=0.02):
        self.teta = np.ones((self.X.shape[1],1))
        y_pred = np.ones_like(self.y)
        cons = lr/len(self.X)
        for _ in range(iteraciones + 1):
            self.teta -= cons * self.X.T@(y_pred - self.y)
            try:
                y_pred = self.X@self.teta
            except ValueError:
                print(f'Dimensiones incorrectas, trabajando en ello...')
                y_pred = self.X@self.teta.reshape(-1,1)
        return self
                
    def predecir(self,**kargs):
        if isinstance(kargs['X'],np.ndarray):
            try:
                return kargs['X']@self.teta
            except ValueError as ex:
                print(f'Dimensiones incorrectas')
        else:
            print(f'¡La entrada debe ser un array!')
    @abstractmethod
    def PHI(self):
        ...

In [ ]:
from itertools import starmap,cycle
class rbf(Regresion):
    """
    Regresión con funciones base radiales
    """
    def __init__(self, X: np.ndarray, y: np.ndarray,mu:int=10,sigma:int=1) -> None:
        super().__init__(X, y)
        self.mu = mu
        self.sigma = sigma
    def PHI(self) -> np.ndarray:
        filas,columnas = self.X.shape
        gauss = lambda x,mu,sig=self.sigma: np.exp(-(1/2*sig**2)*(x-mu)**2)
        rango = np.linspace(-2,2,self.mu)
        phi_matriz = np.repeat(self.X,len(rango))
        phi_matriz = phi_matriz.reshape(filas,columnas*len(rango))
        for f in np.arange(0,filas):
            phi_matriz[f,:] = [i for i in starmap(gauss,zip(phi_matriz[f,:],cycle(rango)))]
        self.X = np.hstack((np.ones((filas,1)),phi_matriz))
        return self

class poly(Regresion):
    """
    Regresión con términos polinomiales
    """
    def __init__(self, X: np.ndarray, y: np.ndarray,grado:int) -> None:
        self.grado = grado
        super().__init__(X, y)
    def PHI(self) -> np.ndarray:
        filas,columnas = self.X.shape
        phi_matriz = np.repeat(self.X,self.grado)
        phi_matriz = phi_matriz.reshape(filas,columnas*self.grado)
        rango = np.arange(1,self.grado+1)
        for f in np.arange(0,filas):
            phi_matriz[f,:] = [i for i in starmap(pow,zip(phi_matriz[f,:],cycle(rango)))]
        self.X = np.hstack((np.ones_like(self.y),phi_matriz))
        return self

In [ ]:
class Red(Regresion):
    def __init__(self, X: np.ndarray, y: np.ndarray,neuronas:int) -> None:
        super().__init__(X, y)
        self.neuronas = {'entrada':self.X.shape[1],'oculta':neuronas}
        self.neuronas['Teta1'] = np.random.normal(size=(self.neuronas['oculta'],self.neuronas['entrada']))
        self.neuronas['bias1'] = np.random.normal(size=(self.neuronas['oculta'],1))
        self.neuronas['Teta2'] = np.random.normal(size=(1,self.neuronas['oculta']))
        self.neuronas['bias2'] = np.random.normal(size=(1))
    
    def RELU(self,Z):
        return np.maximum(Z, 0)
    def RELU_der(self,Z):
        return Z>0

    def PHI(self):
        #Capa entrada
        try:
            self.neuronas['Z1'] = self.neuronas['Teta1']@self.X.T 
        except ValueError as e:
            print(f'Las dimensiones no son compatibles {e.args}')
        self.neuronas['a2'] = self.RELU(self.neuronas['Z1']) + self.neuronas['bias1']

        #Capa de salida
        try:
            self.neuronas['Z2'] = self.neuronas['Teta2']@self.neuronas['a2'] 
        except ValueError as e:
            print(f'Las dimensiones no son compatibles {e.args}')
        self.neuronas['a3'] = self.neuronas['Z2'] + self.neuronas['bias2']
        self.neuronas['a3'].shape = self.y.shape
        return self

    def backprogation(self):
        self.back = {}
        norma = (1/len(self.X))
        #Error de la capa de salida
        self.back['dz3'] = self.neuronas['a3'] - self.y
        try:
            self.back['dteta2'] = self.neuronas['a2']@self.back['dz3'] * norma
        except ValueError:
            print(f'Las dimensiones no son compatibles para dteta2')
        self.back['dbias2'] = np.sum(self.back['dz3']) * norma

        #Error de la capa oculta
        try:
            self.back['dz2'] = (self.back['dz3']@self.neuronas['Teta2']).T * self.RELU_der(self.neuronas['a2'])
        except ValueError:
            print(f'Las dimensiones no son compatibles para dz2')
        try:
            self.back['dteta1'] = self.back['dz2']@self.X * norma
        except ValueError:
                print(f'Las dimensiones no son compatibles para dteta1')
        self.back['dbias1'] = np.sum(self.back['dz2'],axis=1) * norma

        self.back['dteta2'].shape = self.neuronas['Teta2'].shape
        self.back['dteta1'].shape = self.neuronas['Teta1'].shape
        self.back['dbias1'].shape = self.neuronas['bias1'].shape

        return self
    
    def actualizar(self,lr):
        self.neuronas['Teta1'] -= lr*self.back['dteta1']
        self.neuronas['bias1'] -= lr*self.back['dbias1']
        self.neuronas['Teta2'] -= lr*self.back['dteta2']
        self.neuronas['bias2'] -= lr*self.back['dbias2']

In [ ]:
#Escalamiento del dato de entrada y r2_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#X_escalado = scaler.fit_transform(registros['NPHI'].reshape(-1,1))

In [ ]:
#Generar pseudo-registro de permeabilidad
r = rbf(datos_duros['Poro'].reshape(-1,1),np.log(datos_duros['Perme']))
r.X = np.hstack((np.ones_like(r.y),r.X))
r.descenso(800,1.1)
fig = go.Figure(go.Scatter(x=datos_duros['Poro'],y=np.log(datos_duros['Perme']),mode='markers'))
fig.add_trace(go.Scatter(x=datos_duros['Poro'],y=(r.X@r.teta).flatten()))
#fig.show()
#np.hstack((np.ones((183,1)),registros['PhiE'].reshape(-1,1)))@r.teta
phit = np.exp(r.teta[0])*np.exp(r.teta[1]*registros['PhiE'])

In [ ]:
#Para usar más de un registro
X_escalado = scaler.fit_transform(rfn.structured_to_unstructured(registros[['NPHI','RHOB','DT']]))

In [ ]:
#RegresiónRBF
regre_rbf = rbf(X_escalado,np.log(GR_escalado),mu=35,sigma=2.5)
regre_rbf.PHI().descenso(800,0.070)
r2_score(regre_rbf.y,regre_rbf.X@regre_rbf.teta)

0.9337555370552724

In [ ]:
#RegresiónLineal (Añade el término independiente)
regre_lin = rbf(X_escalado,np.log(GR_escalado))
regre_lin.X = np.hstack((np.ones_like(regre_lin.y),regre_lin.X))
regre_lin.descenso(800,0.050)
r2_score(regre_lin.y,regre_lin.X@regre_lin.teta)

0.8699424018061057

In [ ]:
#RegresiónPolinómica
regre_poli = poly(X_escalado,np.log(GR_escalado),12)
regre_poli.PHI().descenso(800,0.050)
r2_score(regre_poli.y,regre_poli.X@regre_poli.teta)

0.9011341471194236

In [ ]:
#RedNeuronal
red = Red(X_escalado,np.log(GR_escalado),5)
for _ in range(700):
  red.PHI().backprogation().actualizar(0.04)
r2_score(np.log(GR_escalado),red.neuronas['a3'])

0.9238054712161009

In [ ]:
fig = make_subplots(rows=1,cols=4,shared_yaxes=True,subplot_titles=('Regresión Polinómica','Regresión Lineal','Regresión RBF','Red Neuronal'))
line=dict(width=2,dash='dashdot')
for idx,modelo in enumerate([regre_poli,regre_lin,regre_rbf]):
  x = np.exp(modelo.X@modelo.teta)
  fig.add_trace(go.Scatter(x=GR_escalado,y=registros['DEPTH'],name='Permeabilidad real',showlegend=False,legendgroup='group2',line_color='red'),row=1,col=idx+1)
  fig.add_trace(go.Scatter(x=x.flatten(),y=registros['DEPTH'],name='Permeabilidad inferida',showlegend=False,legendgroup='group1',line_color='black',line=line),row=1,col=idx+1)
  fig.add_trace(go.Scatter(x=registros['PermNuc'],y=registros['DEPTH'],mode='markers',showlegend=False,legendgroup='group3',name='Nucleo',marker_symbol='triangle-up',marker=dict(color='yellow',size=9,line=dict(width=2))),row=1,col=idx+1)
####
fig.add_trace(go.Scatter(x=GR_escalado,y=registros['DEPTH'],name='Permeabilidad real',showlegend=False,legendgroup='group2',line_color='red'),row=1,col=4)
fig.add_trace(go.Scatter(x=np.exp(red.neuronas['a3']).flatten(),y=registros['DEPTH'],name='Permeabilidad inferida',showlegend=False,line_color='black',line=line,legendgroup='group1'),row=1,col=4)
fig.add_trace(go.Scatter(x=registros['PermNuc'],y=registros['DEPTH'],mode='markers',showlegend=False,legendgroup='group3',name='Nucleo',marker_symbol='triangle-up',marker=dict(color='yellow',size=9,line=dict(width=2))),row=1,col=4)
####
fig['data'][0]['showlegend'] = True
fig['data'][8]['showlegend'] = True
fig['data'][4]['showlegend'] = True
fig['layout']['yaxis']['autorange'] = "reversed"
####
fig.update_xaxes(type='log',range=[-3,5])
fig.update_layout(width=1200,height=800)
fig.show()